In [2]:
!pip install -U sentence-transformers

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 450kB 8.0MB/s 
     |████████████████████████████████| 870kB 43.0MB/s 
     |████████████████████████████████| 1.0MB 43.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5-cp36-none-any.whl size=64942 sha256=96e4735c578083d835dc9ab7f19bc1e19b287dae4c36dc3960142789d7f938d2
  Stored in directory: /root/.cache/pip/wheels/b4/ce/39/5bbda8ac34eb52df8c6531382ca077773fbfcbfb6386e5d66c
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=733ec084b044d1d25ad24eecceaded67af7e4291498918393c8930fc8c36d884
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers sacremoses


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from torch.utils.data import Dataset
import numpy as np 
import torch
import os
import json
import pickle
import random

def pad_tensor(vec, pad, dim):
    pad_size = list(vec.shape)
    pad_size[dim] = pad - vec.size(dim)
    return torch.cat([vec, torch.zeros(*pad_size)], dim=dim)	

use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

100%|██████████| 459M/459M [00:23<00:00, 19.3MB/s]


In [0]:
class SentenceDatasetRoberta(Dataset):
	def __init__(self, load=False, test=False):
		super(SentenceDatasetRoberta, self).__init__()


		self.premise = []
		self.hypothesis = []
		self.label = []
		self.premise_test = []
		self.hypothesis_test = []
		self.label_test = []
		self.max_len = 0
		self.test = test
		self.batch_size = 32

		lab = {
				"neutral" : 0,
				"contradiction" : 1,
				"entailment" : 2
		}


		if(load):
			traindata = '/content/drive/My Drive/Colab Notebooks/mnli_1.0/multinli_1.0_train.jsonl'
			testdata = '/content/drive/My Drive/Colab Notebooks/mnli_1.0/multinli_1.0_dev_matched.jsonl'

			if(test==False):
				with open(traindata, "r") as f:
					i = 0
					for line in f:
						
						jsondata = json.loads(line)

						labl = jsondata["gold_label"]

						if labl == "-":
							continue
						i = i+1
						self.label.append(lab[labl])
						
						sentence = jsondata["sentence1"]
						self.premise.append(sentence)
						sentence = sentence.split(' ')
						if(len(sentence) > self.max_len):
							self.max_len = len(sentence)
						
						sentence = jsondata["sentence2"]
						self.hypothesis.append(sentence)
						sentence = sentence.split(' ')
						if(len(sentence) > self.max_len):
							self.max_len = len(sentence)
						
						
						
						if(i%1000==0):
							print(i)
					'''nums = range(len(self.premise))
					numlist = random.choices(nums, k=40000)
					p = []
					h = []
					l = []
					for i in numlist:
						p.append(self.premise[i])
						h.append(self.hypothesis[i])
						l.append(self.label[i])

					self.premise = model.encode(p)
					self.hypothesis = model.encode(h)
					self.label = l'''
					self.premise = model.encode(self.premise)
					self.hypothesis = model.encode(self.hypothesis)
					file = open('/content/drive/My Drive/Colab Notebooks/pickle/sentences_roberta_train_full.dat','wb+')
					pickle.dump((self.premise, self.hypothesis, self.label), file)
					print("done on training data ", self.max_len, i)
				

			else:
				with open(testdata, "r") as f:
					i = 0;
					for line in f:
						
						jsondata = json.loads(line)

						labl = jsondata["gold_label"]

						if labl == "-":
							continue
						i=i+1
						self.label_test.append(lab[labl])
						
						sentence = jsondata["sentence1"]
						self.premise_test.append(sentence)
						sentence = sentence.split(' ')
						if(len(sentence) > self.max_len):
							self.max_len = len(sentence)
						
						sentence = jsondata["sentence2"]
						self.hypothesis_test.append(sentence)
						sentence = sentence.split(' ')
						if(len(sentence) > self.max_len):
							self.max_len = len(sentence)
						
			
						if(i%1000==0):
							print(i)
							
					self.premise_test = model.encode(self.premise_test)
					self.hypothesis_test = model.encode(self.hypothesis_test)
					file = open('/content/drive/My Drive/Colab Notebooks/pickle/sentences_roberta_test.dat','wb+')
					pickle.dump((self.premise_test, self.hypothesis_test, self.label_test), file)
					print("done on test data ", self.max_len, i)
		else:
			if(self.test):
				file = open('/content/drive/My Drive/Colab Notebooks/pickle/sentences_roberta_test.dat', 'rb+')
				self.premise_test, self.hypothesis_test, self.label_test = pickle.load(file)
				file.close()
			else:
				file = open('/content/drive/My Drive/Colab Notebooks/pickle/sentences_roberta_train_full.dat', 'rb+')
				self.premise, self.hypothesis, self.label = pickle.load(file)
				file.close()

	def __len__(self):
		if(self.test):
			return 9815
		else:
			return 392700

	def __getitem__(self, idx):	
		
		if(self.test):
			premise, hypothesis, label = self.premise_test[idx], self.hypothesis_test[idx], self.label_test[idx] #torch.tensor(self.premise_test[idx]).type(torch.FloatTensor), torch.tensor(self.hypothesis_test[idx]).type(torch.FloatTensor), torch.tensor(self.label_test[idx]).type(torch.LongTensor)
		else:
			premise, hypothesis, label = self.premise[idx], self.hypothesis[idx], self.label[idx] #torch.tensor(self.premise[idx]).type(torch.FloatTensor), torch.tensor(self.hypothesis[idx]).type(torch.FloatTensor), torch.tensor(self.label[idx]).type(torch.LongTensor)
		'''p = []
		p.append(premise)
		premise = model.encode(p)[0]
		h = []
		h.append(hypothesis)
		hypothesis = model.encode(h)[0]'''

		premise, hypothesis, label = torch.tensor(premise).type(torch.FloatTensor), torch.tensor(hypothesis).type(torch.FloatTensor), torch.tensor(label).type(torch.LongTensor)
						
		#premise = pad_tensor(premise, 80, 0)
		#hypothesis = pad_tensor(hypothesis, 80, 0)
		return premise, hypothesis, label



In [7]:
#SentenceDatasetRoberta(load=True, test=True)
SentenceDatasetRoberta(load=True, test=False)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class LinearLayer(nn.Module):
	def __init__(self, batch_size=512, embedding_length=768, num_labels=3):
		super(LinearLayer, self).__init__()

		self.fc = nn.Linear(embedding_length*2, 3)

	def forward(self, premise, hypothesis):
		rep = torch.cat([premise, hypothesis], 1)
		return self.fc(rep)

In [11]:
import os
import gc
import torch
import argparse
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import IPython

parser = argparse.ArgumentParser(description='Experiment RobertaSTS Embeddings')
parser.add_argument('--lr', default=1e-4, type=float, help='learning rate') 
parser.add_argument('--batch_size', default=1024, type=int) 
parser.add_argument('--epochs', '-e', type=int, default=200, help='Number of epochs to train.')
parser.add_argument('--preparedata', type=int, default=1)

args = parser.parse_args(args=[])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


print('==> Preparing data..')

criterion = nn.CrossEntropyLoss()

print('==> Creating networks..')
llmodel = LinearLayer(batch_size = args.batch_size).to(device)
#llmodel.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/weights/networkrobertasts_train.ckpt"))
params = llmodel.parameters()
optimizer = optim.Adam(params, lr=args.lr, weight_decay=1e-5)

print('==> Loading data..')
trainset = SentenceDatasetRoberta()
testset = SentenceDatasetRoberta(test = True)

def train_roberta(currepoch, epoch):
    dataloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True)
    dataloader = iter(dataloader)
    print('\n=> Epoch: %d' % currepoch)
    
    train_loss, correct, total = 0, 0, 0

    for batch_idx in range(len(dataloader)):
        premise, hypothesis, label = next(dataloader)
        if(batch_idx==len(dataloader)-1):
          continue
        premise, hypothesis, label = premise.to(device), hypothesis.to(device), label.to(device)
        optimizer.zero_grad()
        y_pred = llmodel(premise, hypothesis)

        loss = criterion(y_pred, label)
        #print(y_pred, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = y_pred.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        with open("/content/drive/My Drive/Colab Notebooks/logs/robertasts_train_loss.log", "a+") as lfile:
            lfile.write("{}\n".format(train_loss / total))

        with open("/content/drive/My Drive/Colab Notebooks/logs/robertasts_train_acc.log", "a+") as afile:
            afile.write("{}\n".format(correct / total))

        del premise
        del hypothesis
        del label
        gc.collect()
        torch.cuda.empty_cache()
        torch.save(llmodel.state_dict(), '/content/drive/My Drive/Colab Notebooks/weights/networkrobertasts_train.ckpt')
        with open("/content/drive/My Drive/Colab Notebooks/information/robertasts_info.txt", "w+") as f:
            f.write("{} {}".format(currepoch, batch_idx))
        out.update(IPython.display.Pretty('Batch: [%d/%d], Loss: %.3f, Train Loss: %.3f , Acc: %.3f%% (%d/%d)' % (batch_idx, len(dataloader), loss.item(), train_loss/(batch_idx+1), 100.0*correct/total, correct, total)))
    torch.save(llmodel.state_dict(), '/content/drive/My Drive/Colab Notebooks/checkpoints/networkrobertasts_train_epoch_{}.ckpt'.format(currepoch + 1))
    print('\n=> Classifier Network : Epoch [{}/{}], Loss:{:.4f}'.format(currepoch, epoch, train_loss / len(dataloader)))

def test_roberta(currepoch, epoch):
    dataloader = DataLoader(testset, batch_size=args.batch_size, shuffle=True)
    dataloader = iter(dataloader)
    print('\n=> Testing Epoch: %d' % currepoch)
    
    test_loss, correct, total = 0, 0, 0

    for batch_idx in range(len(dataloader)):
        premise, hypothesis, label = next(dataloader)
        if(batch_idx==len(dataloader)-1):
          continue
          
        premise, hypothesis, label = premise.to(device), hypothesis.to(device), label.to(device)
        
        y_pred = llmodel(premise, hypothesis)

        loss = criterion(y_pred, label)

        test_loss += loss.item()
        _, predicted = y_pred.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        with open("/content/drive/My Drive/Colab Notebooks/logs/robertasts_test_loss.log", "a+") as lfile:
            lfile.write("{}\n".format(test_loss / total))

        with open("/content/drive/My Drive/Colab Notebooks/logs/robertasts_test_acc.log", "a+") as afile:
            afile.write("{}\n".format(correct / total))

        del premise
        del hypothesis
        del label
        gc.collect()
        torch.cuda.empty_cache()
        out.update(IPython.display.Pretty('Batch: [%d/%d], Loss: %.3f, Test Loss: %.3f , Acc: %.3f%% (%d/%d)' % (batch_idx, len(dataloader), loss.item(), test_loss/(batch_idx+1), 100.0*correct/total, correct, total)))

    print('\n=> Classifier Network Test: Epoch [{}/{}], Loss:{:.4f}'.format(currepoch, epoch, test_loss / len(dataloader)))


==> Preparing data..
==> Creating networks..
==> Loading data..


In [0]:
out = display(IPython.display.Pretty('Starting'), display_id=True)
#test_roberta(0, args.epochs)
print('==> Training starts..')
for epoch in range(args.epochs):
    train_roberta(epoch, args.epochs)
    test_roberta(epoch, args.epochs)

Batch: [10/384], Loss: 0.817, Train Loss: 0.840 , Acc: 61.923% (6975/11264)

==> Training starts..

=> Epoch: 0

=> Classifier Network : Epoch [0/200], Loss:0.9159

=> Testing Epoch: 0

=> Classifier Network Test: Epoch [0/200], Loss:0.8114

=> Epoch: 1

=> Classifier Network : Epoch [1/200], Loss:0.8640

=> Testing Epoch: 1

=> Classifier Network Test: Epoch [1/200], Loss:0.8066

=> Epoch: 2

=> Classifier Network : Epoch [2/200], Loss:0.8569

=> Testing Epoch: 2

=> Classifier Network Test: Epoch [2/200], Loss:0.8028

=> Epoch: 3

=> Classifier Network : Epoch [3/200], Loss:0.8531

=> Testing Epoch: 3

=> Classifier Network Test: Epoch [3/200], Loss:0.7986

=> Epoch: 4

=> Classifier Network : Epoch [4/200], Loss:0.8505

=> Testing Epoch: 4

=> Classifier Network Test: Epoch [4/200], Loss:0.7983

=> Epoch: 5

=> Classifier Network : Epoch [5/200], Loss:0.8487

=> Testing Epoch: 5

=> Classifier Network Test: Epoch [5/200], Loss:0.7974

=> Epoch: 6

=> Classifier Network : Epoch [6/200], Loss:0.8472

=> Testing Epoch: 6

=> Classifier Network Test: Epoch [6/20